# 네이버 블로그 크롤링 - 제주맛집

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote ## 검색맛집이 들어가기 때문에 변환해주는게 필요하다. 

In [4]:
url = f'https://search.naver.com/search.naver?where=view&sm=tab_jum&query={quote("제주맛집")}'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
time.sleep(2)

In [7]:
from selenium.webdriver.common.keys import Keys

body = driver.find_element_by_tag_name('body')
for _ in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [8]:
soup = BeautifulSoup(driver.page_source)
lis = soup.select('.bx._svp_item')
len(lis)

110

# 제목 

In [11]:
li = lis[0]
title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
title

'만족뿐인 제주 맛집 공유'

날짜 

In [12]:
date = li.select_one('.sub_time.sub_txt').get_text().strip()
date

'2021.11.19.'

# 내용

In [13]:
desc = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
desc

'지난 주말 친구들과 제주여행을 떠났다가 이름알려진 제주 맛집에 갔다왔어요. 전복뚝배기부터 고등어조림,쌈밥 등 풍성한 음식에 아름다운 바다풍경까지 감상할 수 있었던 식당으로 만족스러웠던 곳이랍니다. 1....'

# 하이퍼링크

In [16]:
href = li.select_one('.total_dsc._cross_trigger')['href']


'https://blog.naver.com/tok4/222573156818'

- 날짜처리|

In [17]:
from datetime import datetime, timedelta

today = datetime.now()
today

datetime.datetime(2021, 12, 8, 10, 44, 22, 740156)

In [18]:
today.year,today.month,today.day

(2021, 12, 8)

In [20]:
# 어제
date = today - timedelta(1)
f'{date.year}.{date.month}.{date.day}'

'2021.12.7'

In [22]:
# 6일 전 
date = today - timedelta(6)
f'{date.year}.{date.month}.{date.day}'
13시간 전
date = today - timedelta(hours=13)
f'{date.year}.{date.month}.{date.day}'
45분전
date = today - timedelta(minutes=45)
f'{date.year}.{date.month:02d}.{date.day:02d}'

'2021.12.2'

In [190]:
# s = input()
def convert_date(s):
   if len(s) == 11:
      return s           
   if '어제' in s:
      date = datetime.now()- timedelta(1)
   if '시간' in s:
      date = datetime.now() - timedelta(hours=int(s.replace('시간 전','')))
   if '일' in s:
      date = datetime.now()- timedelta(int(s.replace('일 전','')))
   if '분' in s:
      date = datetime.now() - timedelta(minutes=int(s.replace('분 전','').split()))

   return f'{date.year}.{date.month:02d}.{date.day:02d}'


In [ ]:
s = '1시간전'
convert_date(s)

In [191]:
lines = []
for li in lis:
    title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
    date = li.select_one('.sub_time.sub_txt').get_text().strip()
    desc = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
    href = li.select_one('.total_dsc._cross_trigger')['href']
    lines.append([title,convert_date(date),desc,href])


In [192]:
df = pd.DataFrame(lines,columns=['제목','날짜','내용','href'])
df = df[['제목','날짜','내용']]

In [195]:
df.to_csv('네이버-제주맛집.csv',index=False)

In [196]:
df

,제목,날짜,내용
0,만족뿐인 제주 맛집 공유,2021.11.19.,지난 주말 친구들과 제주여행을 떠났다가 이름알려진 제주 맛집에 갔다왔어요. 전복뚝배...
1,또 생각났던 제주 맛집,2021.12.02,제가 시킨 제주 비빔국수는 갖은 야채들이 그릇에 알록달록 예쁘게 가득 채워져 있었는...
2,북적였던 제주 맛집 리스트,2021.11.13.,무엇보다 비린내 한번 느껴지지 않아 배가 부른 상태였음에도 쉽사리 남길 수가 없었던...
3,감격했던 제주 맛집 소개,2021.11.15.,얼마 전 친구들과 제주로 여행갔다가 현지인께 추천받은 제주 맛집에 다녀왔어요. 갓 ...
4,제주도 가볼만한곳 제주 맛집,2021.12.01,제주도 가볼만한곳 제주 맛집은 깔끔한 메뉴가 푸짐하게 나오기로 소문 난 곳이라 부담...
...,...,...,...
105,제쇼 12월 제주맛집 공유상 진행합니다.^^,2021.12.06,[제쇼 12월 제주맛집 공유상 ] 회원님들의 제주맛집 또는 제주카페 정보를 공유해주...
106,만족했던 소노벨 제주 맛집,2021.10.19.,맛도 퀄리티도 좋았던 곳이라 평소에 족발이나 보쌈 좋아하셨다면 소노벨 제주 맛집 여...
107,[제주맛집] 사장님 귀는 당나귀 귀 정호영쉐프가 운영하는 제주 교래리 우동카덴,2021.11.27.,그렇지만 아주 특별한 맛은 아닌것 같음. 우동 그릇이 엄청 크고 무거움(세수대야 수...
108,풍요로운 제주 맛집 추천,2021.08.15.,이어서 등장만으로도 이목을 집중시킨 제주 맛집 추천의 제주달빛좋은밤이 야심차게 마련...


In [198]:
driver.close()